介绍一些pandas的高级常用操作

数据读取  
header指出哪一行是列名  
na_values哪一个标识表示NaN值  

In [1]:
import pandas as pd
data = pd.read_csv('data.csv',header=0, na_values=["NA"])
print(data)
print(data.dtypes)


   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000
NumRooms    float64
Alley        object
Price         int64
dtype: object


筛选出dtyoes为数值类型的列(Series)，返回一个拷贝的DataFrame  

In [2]:
num_data=data.select_dtypes(include=["number"])
num_data

,NumRooms,Price
0,NaN,127500
1,2.0,106000
2,4.0,178100
3,NaN,140000


提取数值列名（不会拷贝数据！）

In [3]:
numeric_cols = data.select_dtypes(include=["number"]).columns
numeric_cols

Index(['NumRooms', 'Price'], dtype='object')

填充NaN值

In [4]:
means=data[numeric_cols].mean()
data[numeric_cols]=data[numeric_cols].fillna(means)
data

,NumRooms,Alley,Price
0,3.0,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,3.0,NaN,140000


独热编码（One-Hot Encoding），并额外处理缺失值（NaN）  
get_dummies输入数据是array或者DataFrame  
dummy_na处理nan值

输入是array时，返回一个pandas的DataFrame,形状为(array.size,array中不同的值的个数)
把array中所有不同的内容提取出来作为列的标签  
标签内容在array中的索引值（可能不止出现过一次），映射到dataframe的该标签列所对应的index就是true,其余值为false

In [5]:
import numpy as np
color_data=np.array(["Red", "Green", None, "Green"])
dummies = pd.get_dummies(color_data,dummy_na=True)
print(color_data)
print(dummies)

['Red' 'Green' None 'Green']
   Green    Red    NaN
0  False   True  False
1   True  False  False
2  False  False   True
3   True  False  False


In [6]:
data = pd.DataFrame({
    "Color": ["Red", "Blue", None, "Blue"],
    "Size": [np.nan, "M", "L", np.nan],
    "Price": [10, 20, 30, 40]  # 数值列不会被编码
})
print(data)
encoded_data = pd.get_dummies(data, dummy_na=True)
print(encoded_data)

  Color Size  Price
0   Red  NaN     10
1  Blue    M     20
2  None    L     30
3  Blue  NaN     40
   Price  Color_Blue  Color_Red  Color_nan  Size_L  Size_M  Size_nan
0     10       False       True      False   False   False      True
1     20        True      False      False   False    True     False
2     30       False      False       True    True   False     False
3     40        True      False      False   False   False      True


pandas与numpy的互转  
Series、DataFrame 转NumPy数组  to_numpy  

In [7]:
df = pd.DataFrame({"A": [1, 2, 3], "B": ["x", "y", "z"]})
# 方法1: .values (旧方法，仍可用)
numpy_array1 = df.values
# 方法2: .to_numpy() (推荐，更明确)
numpy_array2 = df.to_numpy()
print(numpy_array2)

[[1 'x']
 [2 'y']
 [3 'z']]


In [8]:
s = pd.Series([1, 2, 3])
numpy_array = s.to_numpy()  # 或 s.values
print(numpy_array)  # 输出: [1 2 3]

[1 2 3]


 NumPy 数组转 DataFrame  
 直接用numpy新建df  
 直接用numpy新建Series

In [9]:
arr_df = np.array([[1, "x"], [2, "y"], [3, "z"]])
# 转换为 DataFrame
df = pd.DataFrame(arr_df, columns=["A", "B"])
print(df)

   A  B
0  1  x
1  2  y
2  3  z


In [10]:
arr_s = np.array([1, 2, 3])
s = pd.Series(arr_s)
print(s)

0    1
1    2
2    3
dtype: int64


构造tensor  
只能用numpy构造，所以一般先用pd读取数据处理完后to_numpy,再转tensor  
注意tensor只能是数值类型，字符串需要进行处理  
as_tensor可传入tensor、numpy、list、tuple

In [12]:
import torch
input=pd.get_dummies(df)
print(input)
ts_a=torch.from_numpy(input.to_numpy())
ts_b=torch.as_tensor(input.values.tolist(),device="cuda:0")
print(input.values)
print(ts_a)
print(ts_b)


     A_1    A_2    A_3    B_x    B_y    B_z
0   True  False  False   True  False  False
1  False   True  False  False   True  False
2  False  False   True  False  False   True
[[ True False False  True False False]
 [False  True False False  True False]
 [False False  True False False  True]]
tensor([[ True, False, False,  True, False, False],
        [False,  True, False, False,  True, False],
        [False, False,  True, False, False,  True]])
tensor([[ True, False, False,  True, False, False],
        [False,  True, False, False,  True, False],
        [False, False,  True, False, False,  True]], device='cuda:0')
